In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from six.moves import range
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

# Setup Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
##get the data set with clean reviews
train = pd.read_csv("clean_train_reviewsbg.csv")

In [3]:
#import the word2vec model with gensim
import gensim

In [4]:
#load word2vec
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)


C:\Users\ngmat\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
#change reviews to vectors space anf get the average vector for each review
def review_to_vector(review):
    review_array = review.split()
    wordvectors = []
    for word in review_array:
        try:
            vector = model.get_vector(word)
            wordvectors.append(vector)
        except KeyError:
            pass
    avg_word_vector = np.mean(np.array(wordvectors), axis=0)
    return avg_word_vector

In [6]:
cleanvector = []
for review in train['cleanReview']:
    cleanvector.append(review_to_vector(review))

In [7]:
X_emreviews = np.array(cleanvector, dtype=np.float)
X_emreviews.shape

(25000, 300)

In [8]:
print(X_emreviews[1])

[ 0.03917618  0.07527008  0.00668135  0.12237892 -0.05514144  0.01250296
  0.04666138 -0.04853227  0.06809974  0.06034899 -0.0264349  -0.13563843
 -0.02281494  0.04036713 -0.0774643   0.1085742   0.03467331  0.12186289
  0.01463499 -0.05926056 -0.03535689  0.07528882  0.02703972  0.00123997
  0.04615135 -0.03957943 -0.09244518  0.0536674   0.00888844 -0.00864553
 -0.06121502 -0.01299958 -0.01968536  0.03419232  0.05326509  0.0030509
  0.03214207  0.04327729  0.05265088  0.09054108  0.11198883 -0.01156807
  0.10114269  0.02941818 -0.02399211 -0.05242996 -0.06253014 -0.00025215
  0.03833618 -0.01486816 -0.01893005  0.03220692 -0.04692974  0.01528053
  0.02531347  0.02246552 -0.04301932 -0.09034767  0.02570829 -0.07500491
  0.01776581  0.09853458 -0.09504414 -0.08185279  0.0098011  -0.03730826
 -0.1026289   0.06098423 -0.02648797  0.07591248  0.0409565  -0.00612602
  0.07162933 -0.02266569 -0.12598877 -0.09535122  0.02087936  0.06742859
  0.06781578  0.10706339 -0.02486682 -0.02205276  0.

In [9]:
#train test split with features as vectors and the sentiment score 
train_features, test_features, train_labels, test_labels = train_test_split(X_emreviews, train['sentiment'], test_size = 0.33, random_state = 42)

In [10]:
#set up SVC fit and predict
from sklearn.svm import LinearSVC
svc = LinearSVC()

In [11]:
svc.fit(train_features, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [12]:
sen_pred_train = svc.predict(train_features)
sen_pred_test = svc.predict(test_features)


In [13]:
#performance scores below
from sklearn.metrics import accuracy_score
print('accuracy scores training set: ',accuracy_score(train_labels,sen_pred_train))
print('accuracy scores test set: ',accuracy_score(test_labels,sen_pred_test))

accuracy scores training set:  0.8656119402985074
accuracy scores test set:  0.8578181818181818


In [15]:
from sklearn.metrics import classification_report, confusion_matrix 

In [16]:
print(classification_report(test_labels, sen_pred_test)) 

              precision    recall  f1-score   support

           0       0.86      0.85      0.86      4105
           1       0.85      0.86      0.86      4145

   micro avg       0.86      0.86      0.86      8250
   macro avg       0.86      0.86      0.86      8250
weighted avg       0.86      0.86      0.86      8250



In [17]:
print(confusion_matrix(test_labels, sen_pred_test)) 

[[3495  610]
 [ 563 3582]]


In [18]:
#hyperparamter tuning using C and loss paramters
from sklearn.model_selection import GridSearchCV 

In [41]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
             'loss': ['squared_hinge', 'hinge']}  
  

In [42]:
grid = GridSearchCV(LinearSVC(), param_grid, refit = True, verbose = 3) 

In [43]:
print(LinearSVC())

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [44]:
#same as above
grid.fit(train_features, train_labels)

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=0.1, loss=squared_hinge .......................................
[CV]  C=0.1, loss=squared_hinge, score=0.8542263610315186, total=   0.2s
[CV] C=0.1, loss=squared_hinge .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  C=0.1, loss=squared_hinge, score=0.8507970625111947, total=   0.3s
[CV] C=0.1, loss=squared_hinge .......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s


...

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] ..... C=1000, loss=hinge, score=0.7295360917069675, total=  15.7s
[CV] C=1000, loss=hinge ..............................................


C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.2min finished


[CV] ..... C=1000, loss=hinge, score=0.8310943936951459, total=  15.5s


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10, 100, 1000], 'loss': ['squared_hinge', 'hinge']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [45]:
print(grid.best_params_) 

{'C': 1, 'loss': 'squared_hinge'}


In [46]:
print(grid.best_estimator_)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


In [47]:
grid.predict(test_features)

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [48]:
grid_pred_train = grid.predict(train_features)
grid_pred_test = grid.predict(test_features)

In [49]:
print('accuracy scores training set: ',accuracy_score(train_labels,grid_pred_train))
print('accuracy scores test set: ',accuracy_score(test_labels,grid_pred_test))

accuracy scores training set:  0.8656119402985074
accuracy scores test set:  0.8578181818181818


In [51]:
from sklearn.model_selection import cross_val_score

In [52]:
scores = cross_val_score(grid, X_emreviews, train['sentiment'], cv=5)

C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=0.1, loss=squared_hinge .......................................
[CV]  C=0.1, loss=squared_hinge, score=0.8552789442111578, total=   0.3s
[CV] C=0.1, loss=squared_hinge .......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  C=0.1, loss=squared_hinge, score=0.8514851485148515, total=   0.4s
[CV] C=0.1, loss=squared_hinge .......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s


...

[CV] ..... C=1000, loss=hinge, score=0.7435512897420515, total=  19.5s
[CV] C=1000, loss=hinge ..............................................


C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


[CV] ..... C=1000, loss=hinge, score=0.7913291329132913, total=  19.1s
[CV] C=1000, loss=hinge ..............................................


C:\Users\ngmat\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.9min finished


[CV] ...... C=1000, loss=hinge, score=0.845934593459346, total=  18.8s


In [53]:
#cross valdiate to make sure the model is stable
scores = cross_val_score(svc, X_emreviews, train['sentiment'], cv=5)

In [54]:
print(scores)

[0.8574 0.8642 0.858  0.8632 0.8532]
